In [1]:
# 실행 후 다시 실행
!pip install numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.1 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.1 which is incompatible.
albumentations 1.4.13 requires numpy>=1.24.4, but you have numpy 1.23.1 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.23.1 which is incompatible.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11

In [1]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661654 sha256=c68e97891af866996fe98c213d4306a81ba6e239387d1cfebc1112595c3dd96b
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ezm759iv/kobert-tokenizer_c591875f476b4c22856a65e49b81d4ed
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ezm759iv/kobert-tokenizer_c591875f476b4c22856a65e49b81d4ed
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=85380891eca972c7648791bc29f6a2006e7300fe5358ef263c416c8a71f2e8a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-uqdwc06i/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
#구글 드라이브 import
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/유런 24 여름 방학 프로젝트/eda&전처리/preprocessing_final.csv")

In [ ]:
#feature label 나누기
data_list = []
for title, label in zip(df['processed_title'], df['label']):
  data = []
  data.append(title)
  data.append(str(label))

  data_list.append(data)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 128
warmup_ratio = 0.1
max_grad_norm = 1

In [ ]:
#train & val 데이터로 나누기
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
df_train = [[str(text), int(label)] for text, label in df_train]
df_val = [[str(text), int(label)] for text, label in df_val]

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(df_train, 0, 1, tok, vocab, max_len, True, False)
data_val = BERTDataset(df_val,0, 1, tok, vocab,  max_len, True, False)

**dataloader**

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

def ret_dataloader(batch_size):
    train_dataloader = DataLoader(
                data_train,  # The training samples.
                sampler = RandomSampler(data_train), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    validation_dataloader = DataLoader(
                data_val, # The validation samples.
                sampler = SequentialSampler(data_val), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    return train_dataloader,validation_dataloader

**Bert Classifier**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert, # bertmodel 입력받음.
                 dr_rate,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                              attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

**optimizer**

In [ ]:
def ret_optim(model, lr_rate):
    optimizer = AdamW(model.parameters(),
                      lr = lr_rate,
                      eps = 1e-8
                    )
    return optimizer

**scheduler**

In [ ]:
def ret_scheduler(train_dataloader, optimizer, epochs):
    # Total number of training steps is [number of batches] x [number of epochs].
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs
    warmup_step = int(total_steps * warmup_ratio)

    # Create the learning rate scheduler.
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps = warmup_step,
                                                num_training_steps = total_steps)
    return scheduler

In [ ]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## Training

In [ ]:
def train(epochs, lr_rate, batch_size, dr_rate):
    # model initialize
    model = BERTClassifier(bertmodel, dr_rate=dr_rate)
    model.to(device)

    train_dataloader,validation_dataloader = ret_dataloader(batch_size)
    optimizer = ret_optim(model, lr_rate)
    scheduler = ret_scheduler(train_dataloader, optimizer, epochs)
    loss_fn = nn.CrossEntropyLoss()

    # We'll store a number of quantities such as training and validation loss,
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        model.train()

        # For each batch of training data...
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
            optimizer.zero_grad()

            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)

            loss = loss_fn(out, label)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)


        print("")
        print("  Average training loss: {0:.4f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(validation_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            with torch.no_grad() :
                out = model(token_ids, valid_length, segment_ids)

                loss = loss_fn(out, label)

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += calc_accuracy(out, label)

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.4f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    return model

In [ ]:
model = train(epochs=5, lr_rate=2e-5, batch_size=8, dr_rate=0.3)


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.4993
  Training epcoh took: 0:02:44

Running Validation...
  Accuracy: 0.8667
  Validation Loss: 0.3816
  Validation took: 0:00:12

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.3330
  Training epcoh took: 0:02:46

Running Validation...
  Accuracy: 0.8882
  Validation Loss: 0.3231
  Validation took: 0:00:12

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.2703
  Training epcoh took: 0:02:46

Running Validation...
  Accuracy: 0.8898
  Validation Loss: 0.3680
  Validation took: 0:00:12

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.2183
  Training epcoh took: 0:02:46

Running Validation...
  Accuracy: 0.9089
  Validation Loss: 0.3296
  Validation took: 0:00:12

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.1866
  Training epcoh took: 0:02:46

Running Validation...
  Accuracy: 0.9094
  Validation Loss: 0.3561
  Validation took

In [ ]:
# save model

torch.save(model.state_dict(), '/content/drive/MyDrive/유런 24 여름 방학 프로젝트/BERT/KoBERT/kobert_plain.pth')

**llrd 적용**

In [ ]:
def ret_optim_llrd(model, lr_rate):
    # LLRD 적용
    optimizer_grouped_parameters = [
        {'params': model.bert.encoder.layer[:6].parameters(), 'lr': lr_rate * 0.1},
        {'params': model.bert.encoder.layer[6:12].parameters(), 'lr': lr_rate * 0.5},
        {'params': model.classifier.parameters(), 'lr': lr_rate}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, eps=1e-8)
    return optimizer

In [ ]:
def train(epochs, lr_rate, batch_size, dr_rate):

    model = BERTClassifier(bertmodel, dr_rate=dr_rate)
    model.to(device)

    train_dataloader,validation_dataloader = ret_dataloader(batch_size)
    optimizer = ret_optim_llrd(model, lr_rate) #llrd_optimizer 적용
    scheduler = ret_scheduler(train_dataloader, optimizer, epochs)
    loss_fn = nn.CrossEntropyLoss()

    # We'll store a number of quantities such as training and validation loss,
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        model.train()

        # For each batch of training data...
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
            optimizer.zero_grad()

            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)

            loss = loss_fn(out, label)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)


        print("")
        print("  Average training loss: {0:.4f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(validation_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            with torch.no_grad() :
                out = model(token_ids, valid_length, segment_ids)

                loss = loss_fn(out, label)

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += calc_accuracy(out, label)

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.4f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    return model

In [ ]:
model = train(epochs=3, lr_rate=5e-5, batch_size=8, dr_rate=0.5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 3 ========
Training...

  Average training loss: 0.2221
  Training epcoh took: 0:02:44

Running Validation...
  Accuracy: 0.9036
  Validation Loss: 0.4186
  Validation took: 0:00:12

======== Epoch 2 / 3 ========
Training...

  Average training loss: 0.1622
  Training epcoh took: 0:02:44

Running Validation...
  Accuracy: 0.9089
  Validation Loss: 0.3760
  Validation took: 0:00:12

======== Epoch 3 / 3 ========
Training...

  Average training loss: 0.1334
  Training epcoh took: 0:02:45

Running Validation...
  Accuracy: 0.9068
  Validation Loss: 0.4107
  Validation took: 0:00:12

Training complete!
Total training took 0:08:49 (h:mm:ss)


In [ ]:
# save model

torch.save(model.state_dict(), '/content/drive/MyDrive/유런 24 여름 방학 프로젝트/BERT/KoBERT/kobert_llrd.pth')